In [1]:
import sys
sys.path.append(r'C:\Users\aburtnerabt\Documents\Continuing Education\Algo Trading\algo_trading\src')

In [9]:
import math
import pandas as pd
from features.build_features import feature_pipeline

In [3]:
data = feature_pipeline(r'C:\Users\aburtnerabt\Documents\Continuing Education\Algo Trading\algo_trading\data\raw\A')

C:\Users\aburtnerabt\Documents\Continuing Education\Algo Trading\algo_trading\src\features\build_features.py:68: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.ta.mcgd(append=True)


In [5]:
#make simple percent change target for time frames
def add_aboslute_percent_increase(data):
    data['%_change_adj_close_28'] = data['adj close'].pct_change(periods=28)
    data['%_change_adj_close_21'] = data['adj close'].pct_change(periods=21)
    data['%_change_adj_close_14'] = data['adj close'].pct_change(periods=14)
    data['%_change_adj_close_7'] = data['adj close'].pct_change(periods=7)
    return data

In [7]:
#make log return column, then log return percents
def add_percent_log_increase(data):
    data['log_adj_close'] = data['adj close'].apply(lambda x: math.log10(x))
    data['%_change_log_adj_close_28'] = data['log_adj_close'].pct_change(periods=28)
    data['%_change_log_adj_close_21'] = data['log_adj_close'].pct_change(periods=21)
    data['%_change_log_adj_close_14'] = data['log_adj_close'].pct_change(periods=14)
    data['%_change_log_adj_close_7'] = data['log_adj_close'].pct_change(periods=7)
    return data

In [8]:
first_date = data.index[0]
data.loc[first_date]['adj close']

65.06788635253906

In [10]:
second_date = first_date + pd.DateOffset(days=28)
data.loc[first_date:second_date]['adj close']

date
2019-05-29    65.067886
2019-05-30    64.671501
2019-05-31    64.826195
2019-06-03    64.768173
2019-06-04    65.696342
2019-06-05    66.083069
2019-06-06    66.866203
2019-06-07    67.214256
2019-06-10    67.958740
2019-06-11    67.765358
2019-06-12    67.958740
2019-06-13    67.968391
2019-06-14    67.649338
2019-06-17    67.958740
2019-06-18    69.525002
2019-06-19    70.240456
2019-06-20    70.888222
2019-06-21    70.907570
2019-06-24    70.714211
2019-06-25    70.105095
2019-06-26    69.747383
Name: adj close, dtype: float64

In [14]:
draw_down = 66
int(any(map(lambda x: x<draw_down, data.loc[first_date:second_date]['adj close'])))

1

In [16]:
len(data.index)


1163

In [17]:
len(data.index[:-10])

1153

In [29]:
def encode_outcome_target(df, tgt_column, drawdown_pct, increase_pct, days):
    target_values = []
    #for each date in index get value 28 days ahead
    for date in df.index[:-days]:
        current_val = df.loc[date][tgt_column]
        min_future_val = current_val * (1+increase_pct)
        drawdown_amount = current_val * (1-drawdown_pct)
        #make sure values in between date and tgt date 
        #dont drop below drawdown pct target    
        end_date = date + pd.DateOffset(days=days)
        returns = df.loc[date:end_date][tgt_column]

        #identify if price drops below floor
        drawdown_indicator = int(any(map(lambda x: x<drawdown_amount, returns)))

        #identify if price goes above target
        return_indicator = int(any(map(lambda x: x>min_future_val, returns)))

        #if below floor return 0, if above target return 1, otherwise return 0
        if drawdown_indicator == 1:
            target = 0
        elif return_indicator == 0:
            target = 0
        else:
            target = 1
        target_values.append(target)
    #add null values for 
    for day in range(days):
        target_values.append(None)
    return target_values

        

In [30]:
test = encode_outcome_target(data, 'adj close', .05, .05, 28)

In [31]:
data['5%_higher_5%_drawdown'] = test

In [32]:
data['5%_higher_5%_drawdown'].sum()

517.0

In [33]:
checker = data[['adj close','5%_higher_5%_drawdown']]

In [4]:
print(data.shape)
print(max(data.index))
data = data.dropna()
print(data.shape)
print(max(data.index))

(1259, 215)
2024-01-09 00:00:00
(1163, 215)
2024-01-09 00:00:00
